In [78]:
from __future__ import division


#https://python-chess.readthedocs.io/en/latest/
import chess, random
from mcts import mcts
from chess import Move, svg
from IPython.display import display


from copy import deepcopy
from mcts import mcts
from functools import reduce
import operator



In [79]:
BLACK = -1
WHITE = 1

class ChessState():
    def __init__(self):
        self.board = chess.Board()
        self.currentPlayer = 1
    def getCurrentPlayer(self):
        # 1 for maximiser, -1 for minimiser
        return self.currentPlayer

    def getLegalActions(self):
        legal_actions = list(self.board.legal_moves)
        result = []
        for a in legal_actions:
            result.append(Action(a.uci()))
        return result

    def generateSuccessor(self, action):
        if action is not None:
        
            newState = deepcopy(self)
            newState.board.push(Move.from_uci(action.uci))
            newState.currentPlayer = -1 * newState.currentPlayer
            return newState
        return self

    def isTerminal(self):
        return self.board.is_game_over()
    
    def isWin(self):
        if self.isTerminal():
            res = self.board.result()
            if res == "1-0":
                return True
            else:
                return False
        else:
            return False
    
    def isLose(self):
        if self.isTerminal():
            return not self.isWin()
        else:
            return False
    
    #DRAWS
    
    def getReward(self):
        # only needed for terminal states
        if self.isTerminal():
            res = self.board.result()
            if res == "1-0":
                return 1
            elif res == "0-1":
                return -1
            else:
                return 0
        else:
            return 0
        
    

In [80]:
class Action():
    def __init__(self, uci):
        self.uci = uci
    def __hash__(self):
        return hash(self.uci)
    def __str__(self):
        return self.uci
    def __repr__(self):     
        return self.uci

In [81]:
from abc import ABC, abstractmethod
class Agent(ABC):
    @abstractmethod
    def getAction(self, possible_actions):
        pass

In [82]:
class RandomAgent(Agent):
    def getAction(self, possible_actions):
        return random.choice(possible_actions)

In [83]:
import random


class ReflexAgent(Agent):
    """
    A reflex agent chooses an action at each choice point by examining
    its alternatives via a state evaluation function.
    The code below is provided as a guide.  You are welcome to change
    it in any way you see fit, so long as you don't touch our method
    headers.
    """

    def getAction(self, gameState):
        """
        You do not need to change this method, but you're welcome to.
        getAction chooses among the best options according to the evaluation function.
        Just like in the previous project, getAction takes a GameState and returns
        some Directions.X for some X in the set {NORTH, SOUTH, WEST, EAST, STOP}
        """
        # Collect legal moves and successor states
        legalMoves = gameState.getLegalActions()

        # Choose one of the best actions
        scores = [self.evaluationFunction(gameState, action) for action in legalMoves]
        bestScore = max(scores)
        bestIndices = [index for index in range(len(scores)) if scores[index] == bestScore]
        chosenIndex = random.choice(bestIndices)  # Pick randomly among the best

        "Add more of your code here if you want to"

        return legalMoves[chosenIndex]

    def evaluationFunction(self, currentGameState, action):
    
        return scoreEvaluationFunction(currentGameState.generateSuccessor(action))
    #         [1, 3, 3, 5, 9, 0]
        


def scoreEvaluationFunction(currentGameState):
    """
    This default evaluation function just returns the score of the state.
    The score is the same one displayed in the Pacman GUI.
    This evaluation function is meant for use with adversarial search agents
    (not reflex agents).
    """
    score = 0
    fen = currentGameState.board.fen()
    piece_values = {'p': -1, 'P': 1, 'k': 0, 'K': 0, 'n': -3, 'N':3, 'b': -3, 'B':3, 'r': -5, 'R':5, 'q': -9, 'Q': 9}
    
    def get_value(char):
        try:
            return piece_values[char]
        except:
            return 0
    for i in range(fen.index(" ")):
        score+=get_value(fen[i])
        
    return score


class MultiAgentSearchAgent(Agent):
    """
    This class provides some common elements to all of your
    multi-agent searchers.  Any methods defined here will be available
    to the MinimaxPacmanAgent, AlphaBetaPacmanAgent & ExpectimaxPacmanAgent.
    You *do not* need to make any changes here, but you can if you want to
    add functionality to all your adversarial search agents.  Please do not
    remove anything, however.
    Note: this is an abstract class: one that should not be instantiated.  It's
    only partially specified, and designed to be extended.  Agent (game.py)
    is another abstract class.
    """

    def __init__(self, evalFn=scoreEvaluationFunction, depth='3'):
        self.index = 0  # Pacman is always agent index 0
        self.evaluationFunction = evalFn
        self.depth = int(depth)


class MinimaxAgent(MultiAgentSearchAgent):
    
    def min_value(self,state, depth):
        v = float("inf")
        best_action_indices = []
        actions = state.getLegalActions()

        if state.isWin() or state.isLose():
            return self.evaluationFunction(state), None

        if depth >= self.depth:
            for action_index in range(len(actions)):
                action = actions[action_index]
                successor_state = state.generateSuccessor(action)
                successor_value = self.evaluationFunction(successor_state)
                if successor_value < v:
                    v = successor_value
                    best_action_indices = [action_index]
                if successor_value == v:
                    best_action_indices+=[action_index]
                 
            return v, actions[random.choice(best_action_indices)]
        else:
            for action_index in range(len(actions)):
                action = actions[action_index]
                successor_state = state.generateSuccessor(action)
                successor_value = self.max_value(successor_state, depth + 1)[0]
                if successor_value < v:
                    v = successor_value
                    best_action_indices = [action_index]
                if successor_value == v:
                    best_action_indices+=[action_index]
                    
            return v, actions[random.choice(best_action_indices)]

    def max_value(self,state, depth):
        v = float("-inf")
        best_action_indices = []
        actions = state.getLegalActions()
        if state.isWin() or state.isLose() :
            return self.evaluationFunction(state), None
        if depth >= self.depth :

            for action_index in range(len(actions)):
                action = actions[action_index]
                successor_state = state.generateSuccessor(action)
                successor_value = self.evaluationFunction(successor_state)
                if successor_value > v:
                    v = successor_value
                    best_action_indices = [action_index]
                if successor_value == v:
                    best_action_indices+=[action_index]
            return v, actions[random.choice(best_action_indices)]
        else :
            for action_index in range(len(actions)):
                action = actions[action_index]
                successor_state = state.generateSuccessor(action)
                successor_value = self.min_value(successor_state, depth)[0]
                if successor_value > v:
                    v = successor_value
                    best_action_indices = [action_index]
                if successor_value == v:
                    best_action_indices+=[action_index]
            return v, actions[random.choice(best_action_indices)]
        
    def getAction(self, gameState):
        if gameState.currentPlayer == 1 :
            return self.max_value(gameState, 0)[1]
        if gameState.currentPlayer == -1 :
            return self.min_value(gameState, 0)[1]


In [87]:
class AlphaBetaAgent(MultiAgentSearchAgent):
    """
    Your minimax agent with alpha-beta pruning (question 3)
    """

    def min_value(self,state, depth, alpha, beta):
        v = float("inf")
        best_action_indices = [0]
        actions = state.getLegalActions()
        #actions = random.sample(actions, len(actions)//2)
        #print(len(actions))

        if state.isWin() or state.isLose():
            return self.evaluationFunction(state), None

        if depth >= self.depth:
            for action_index in random.sample(range(len(actions)), k = len(actions)//4):
                action = actions[action_index]
                successor_state = state.generateSuccessor(action)
                successor_value = self.evaluationFunction(successor_state)
                if successor_value < v:
                    v = successor_value
                    best_action_indices = [action_index]
                if successor_value == v:
                    best_action_indices+=[action_index]
                 
            return v, actions[random.choice(best_action_indices)]
        else:
            currentBeta = beta
            for action_index in random.sample(range(len(actions)), k = len(actions)//4):
                action = actions[action_index]
                successor_state = state.generateSuccessor(action)
                successor_value = self.max_value(successor_state, depth + 1, alpha, currentBeta)[0]
                if successor_value < v:
                    v = successor_value
                    best_action_indices = [action_index]
                if successor_value == v:
                    best_action_indices+=[action_index]
                if v < alpha:
                    return v, actions[random.choice(best_action_indices)]
                currentBeta = min(currentBeta, v)
                    
            return v, actions[random.choice(best_action_indices)]

    def max_value(self,state, depth, alpha, beta):
        v = float("-inf")
        best_action_indices = [0]
        actions = state.getLegalActions()
        
        #actions = random.sample(actions, len(actions)//2)
        #print(len(actions))

        if state.isWin() or state.isLose() :
            return self.evaluationFunction(state), None
        if depth >= self.depth :
            for action_index in random.sample(range(len(actions)), k = len(actions)//4):
                action = actions[action_index]
                successor_state = state.generateSuccessor(action)
                successor_value = self.evaluationFunction(successor_state)
                if successor_value > v:
                    v = successor_value
                    best_action_indices = [action_index]
                if successor_value == v:
                    best_action_indices+=[action_index]
    
            return v, actions[random.choice(best_action_indices)]
        else :
            currentAlpha = alpha
            for action_index in random.sample(range(len(actions)), k = len(actions)//4):
                action = actions[action_index]
                successor_state = state.generateSuccessor(action)
                successor_value = self.min_value(successor_state, depth, currentAlpha, beta)[0]
                if successor_value > v:
                    v = successor_value
                    best_action_indices = [action_index]
                if successor_value == v:
                    best_action_indices+=[action_index]
                if v > beta:
                    #WHATTTT
                    return v, actions[random.choice(best_action_indices)]
                currentAlpha = max(currentAlpha, v)
            return v, actions[random.choice(best_action_indices)]
        
    def getAction(self, gameState):
        if gameState.currentPlayer == 1 :
            return self.max_value(gameState, 0, float("-inf"), float("inf"))[1]
        if gameState.currentPlayer == -1 :
            return self.min_value(gameState, 0, float("-inf"), float("inf"))[1]


In [93]:
def gameplay(agent):
    b = ChessState()
#    display(svg.board(b.board, size=100)) 
#     white = True
    for i in range(2):
#     while not b.isTerminal():
        action = agent.getAction(b)
        print("move:", i, action)
        b = b.generateSuccessor(action)
        #display(svg.board(b.board, size=100)) 
    

In [94]:
import time

print("AlphaBeta starting")
agent = AlphaBetaAgent()
start_time = time.time()
gameplay(agent)
print("--- %s seconds ---" % (time.time() - start_time))



AlphaBeta starting
move: 0 b2b4
move: 1 g8h6
--- 31.33649468421936 seconds ---


In [2]:
def alphabeta(position, depth, alpha, beta):
    """Returns a tuple (score, bestmove) for the position at the given depth"""
    if depth == 0 or position.is_checkmate() or position.is_draw():
        return (position.evaluate(), None)
    else: 
        if position.to_move == "white":
            bestmove = None
            for move in position.legal_moves():
                new_position = position.make_move(move)
                score, move = alphabeta(new_position, depth - 1, alpha, beta)
                if score > alpha: # white maximizes her score
                    alpha = score
                    bestmove = move
                    if alpha >= beta: # alpha-beta cutoff
                        break
            return (alpha, bestmove)
        else:
            bestmove = None
            for move in position.legal_moves():
                new_position = position.make_move(move)
                score, move = alphabeta(new_position, depth - 1, alpha, beta)
                if score < beta: # black minimizes his score
                    beta = score
                    bestmove = move
                    if alpha >= beta: # alpha-beta cutoff
                        break
            return (beta, bestmove)

In [ ]:
#https://www.chessprogramming.org/PeSTO%27s_Evaluation_Function
#piece square table, mid game, end game